##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gradient Boosted Trees: Понимание модели

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/estimator/boosted_trees_model_understanding"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотреть на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/boosted_trees_model_understanding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустить в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/estimator/boosted_trees_model_understanding.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Смотреть исходные файлы на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/estimator/boosted_trees_model_understanding.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачать ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Для получения полного пошагового руководства по обучению модели градиентного бустинга обратитесь к [учебному пособию по решающим деревьям](./boosted_trees). В этом уроке вы:

* Узнаете, как интерпретировать модель Boosted Trees *локально* и *глобально*
* Получите представление о том, как модель Boosted Trees обучается на наборе данных

## Как интерпретировать модели Boosted Trees локально и глобально

Локальная интерпретируемость относится к пониманию предсказаний модели на уровне отдельного примера, а глобальная интерпретируемость относится к пониманию модели в целом. Такие методы могут помочь специалистам, практикующим машинное обучение(ML), обнаруживать необъективность модели и ошибки на этапе разработки модели.

Для обеспечения локальной интерпретируемости вы узнаете, как создавать и визуализировать взнос каждого экземпляра модели. Чтобы отличить это от важности признаков, мы называем эти значения направленным вкладом признаков(DFC).

Для глобальной интерпретируемости вы будете извлекать и визуализировать значения признаков на основе усиления, [изменение важности признаков](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf), а также показывать агрегированные DFC.

## Загрузка датасета Титаник
Вы будете использовать набор данных пассажиров Титаника, цель которого(довольно болезненная) - предсказать выживаемость пассажиров с учетом таких характеристик, как пол, возраст, класс и т.д.

In [ ]:
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import clear_output

# Загружаем датасет.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

In [ ]:
import tensorflow as tf
tf.random.set_seed(123)

Описание признаков см. в предыдущем руководстве.

## Создание столбцов признаков, функции input_fn, и обучение оценщика

### Подготовка данных

Создайте столбцы признаков, используя исходные числовые столбцы как есть, а категориальные признаки преобразуйте  в one-hot-encoding столбцы.

In [ ]:
fc = tf.feature_column
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

def one_hot_cat_column(feature_name, vocab):
  return fc.indicator_column(
      fc.categorical_column_with_vocabulary_list(feature_name,
                                                 vocab))
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  # Нужно перобразовать категориальные признаки в числовые с помощью `one-hot-encoding` метода.
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(one_hot_cat_column(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(fc.numeric_column(feature_name,
                                           dtype=tf.float32))

### Создание входного конвейера

Создайте функции ввода, используя метод from_tensor_slices из [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) для чтения данных непосредственно из Pandas.

In [ ]:
# Используйте весь пакет, так как это небольшой набор данных.
NUM_EXAMPLES = len(y_train)

def make_input_fn(X, y, n_epochs=None, shuffle=True):
  def input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
    if shuffle:
      dataset = dataset.shuffle(NUM_EXAMPLES)
    # Для обучения повторите набор данных столько раз, сколько потребуется(n_epochs = None).
    dataset = (dataset
      .repeat(n_epochs)
      .batch(NUM_EXAMPLES))
    return dataset
  return input_fn

# Функции ввода для обучения и оценки.
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, shuffle=False, n_epochs=1)

### Обучение модели

In [ ]:
params = {
  'n_trees': 50,
  'max_depth': 3,
  'n_batches_per_layer': 1,
  # Вы должны установить center_bias=True для получения DFC. 
  # Это заставит модель сделать первоначальный прогноз перед использованием 
  # каких-либо признаков (например, использовать среднее значение меток для регрессии,
  # или логарифм перевесов для классификации при использовании кросс-энтропийной ф-ции потерь).
  'center_bias': True
}

est = tf.estimator.BoostedTreesClassifier(feature_columns, **params)
# Обучение модели
est.train(train_input_fn, max_steps=100)

# Оценка.
results = est.evaluate(eval_input_fn)
clear_output()
pd.Series(results).to_frame()

Для улучшения производительности, если ваши данные умещаются в памяти, мы рекомендуем использовать функцию `boosted_trees_classifier_train_in_memory`. Однако, если время обучения не имеет значения или если у вас очень большой набор данных и вы хотите проводить распределенное обучение, используйте `tf.estimator.BoostedTrees`, показанный выше.
При использовании этого метода не следует разделять на пакеты входные данные, поскольку метод работает со всем набором данных.

In [ ]:
in_memory_params = dict(params)
in_memory_params['n_batches_per_layer'] = 1
# Ин-мемори input_fn не использует пакеты
def make_inmemory_train_input_fn(X, y):
  y = np.expand_dims(y, axis=1)
  def input_fn():
    return dict(X), y
  return input_fn
train_input_fn = make_inmemory_train_input_fn(dftrain, y_train)

# Обучение модели
est = tf.estimator.BoostedTreesClassifier(
    feature_columns, 
    train_in_memory=True, 
    **in_memory_params)

est.train(train_input_fn)
print(est.evaluate(eval_input_fn))

## Интерпритация модели и графики

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns_colors = sns.color_palette('colorblind')

## Локальная интерпритация
Далее вы выведете DFC для объяснения индивидуальных прогнозов, используя подход, описанный в [Palczewska et al](https://arxiv.org/pdf/1312.1121.pdf) и Saabas в [Интерпретация случайных лесов]( http://blog.datadive.net/interpreting-random-forests/) (этот метод также доступен в scikit-learn для Random Forests в пакете [`treeinterpreter`](https://github.com/andosa/treeinterpreter)). 
DFC создаются с помощью:

`pred_dicts = list (est.experimental_predict_with_explanations (pred_input_fn))`

(Примечание: этот метод называется экспериментальным, поэтому стоит учитывать, что мы можем изменить API перед удалением экспериментального префикса.)

In [ ]:
pred_dicts = list(est.experimental_predict_with_explanations(eval_input_fn))

In [ ]:
# Создадим DFC Pandas датафрейм.
labels = y_eval.values
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])
df_dfc = pd.DataFrame([pred['dfc'] for pred in pred_dicts])
df_dfc.describe().T

Хорошим свойством DFC является то, что сумма величин признаков + смещение равна предсказанию для данного примера.

In [ ]:
# сумма DFCs + bias == прогноз.
bias = pred_dicts[0]['bias']
dfc_prob = df_dfc.sum(axis=1) + bias
np.testing.assert_almost_equal(dfc_prob.values,
                               probs.values)

Постройте DFC для отдельного пассажира. Давайте раскрасим график с помощью цветовой кодировки на основе DFC и добавим значения характеристик на рисунке.

In [ ]:
# Код построения графика
def _get_color(value):
    """To make positive DFCs plot green, negative DFCs plot red."""
    green, red = sns.color_palette()[2:4]
    if value >= 0: return green
    return red

def _add_feature_values(feature_values, ax):
    """Display feature's values on left of plot."""
    x_coord = ax.get_xlim()[0]
    OFFSET = 0.15
    for y_coord, (feat_name, feat_val) in enumerate(feature_values.items()):
        t = plt.text(x_coord, y_coord - OFFSET, '{}'.format(feat_val), size=12)
        t.set_bbox(dict(facecolor='white', alpha=0.5))
    from matplotlib.font_manager import FontProperties
    font = FontProperties()
    font.set_weight('bold')
    t = plt.text(x_coord, y_coord + 1 - OFFSET, 'feature\nvalue',
    fontproperties=font, size=12)

def plot_example(example):
  TOP_N = 8 # Покажем топ 8 признаков.
  sorted_ix = example.abs().sort_values()[-TOP_N:].index  # Сортируем по величине.
  example = example[sorted_ix]
  colors = example.map(_get_color).tolist()
  ax = example.to_frame().plot(kind='barh',
                          color=[colors],
                          legend=None,
                          alpha=0.75,
                          figsize=(10,6))
  ax.grid(False, axis='y')
  ax.set_yticklabels(ax.get_yticklabels(), size=14)

  # Добавим значение признаков.
  _add_feature_values(dfeval.iloc[ID][sorted_ix], ax)
  return ax

In [ ]:
# График.
ID = 182
example = df_dfc.iloc[ID]  # Выбираем i-й пример из оценочного датасета.
TOP_N = 8  # Показываем топ 8 признаков.
sorted_ix = example.abs().sort_values()[-TOP_N:].index
ax = plot_example(example)
ax.set_title('Feature contributions for example {}\n pred: {:1.2f}; label: {}'.format(ID, probs[ID], labels[ID]))
ax.set_xlabel('Contribution to predicted probability', size=14)
plt.show()

Значения большей величины имеют большее влияние на предсказание модели. Отрицательные значения указывают на то, что значение функции для данного примера уменьшило прогноз модели, а положительные значения способствуют увеличению прогноза

Вы также можете построить график сравнения примеров DFC со всем распределением, используя график `voilin`.

In [ ]:
# Код построения графика.
def dist_violin_plot(df_dfc, ID):
  # Инициализируем объект графика.
  fig, ax = plt.subplots(1, 1, figsize=(10, 6))

  # Создаем пример датафрейма.
  TOP_N = 8  # Показываем топ 8 признаков.
  example = df_dfc.iloc[ID]
  ix = example.abs().sort_values()[-TOP_N:].index
  example = example[ix]
  example_df = example.to_frame(name='dfc')

  # Добавляем величины полного распределния.
  parts=ax.violinplot([df_dfc[w] for w in ix],
                 vert=False,
                 showextrema=False,
                 widths=0.7,
                 positions=np.arange(len(ix)))
  face_color = sns_colors[0]
  alpha = 0.15
  for pc in parts['bodies']:
      pc.set_facecolor(face_color)
      pc.set_alpha(alpha)

  # Добавляем значения признаков.
  _add_feature_values(dfeval.iloc[ID][sorted_ix], ax)

  # Добавляем локальные вклады.
  ax.scatter(example,
              np.arange(example.shape[0]),
              color=sns.color_palette()[2],
              s=100,
              marker="s",
              label='contributions for example')

  # Разметка
  ax.plot([0,0], [1,1], label='eval set contributions\ndistributions',
          color=face_color, alpha=alpha, linewidth=10)
  legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large',
                     frameon=True)
  legend.get_frame().set_facecolor('white')

  # Форматируем график.
  ax.set_yticks(np.arange(example.shape[0]))
  ax.set_yticklabels(example.index)
  ax.grid(False, axis='y')
  ax.set_xlabel('Contribution to predicted probability', size=14)

Отрисуем график этого примера

In [ ]:
dist_violin_plot(df_dfc, ID)
plt.title('Feature contributions for example {}\n pred: {:1.2f}; label: {}'.format(ID, probs[ID], labels[ID]))
plt.show()

Наконец, сторонние инструменты, такие как [LIME](https://github.com/marcotcr/lime) и [shap](https://github.com/slundberg/shap), также могут помочь понять индивидуальные прогнозы для модели.

## Глобальная важность признаков

Кроме того, вы можете захотеть понять модель в целом, а не изучать отдельные прогнозы. Ниже вы будете вычислять и использовать:

* Важность признаков на основании их силы влияния на прогноз с использованием `est.experimental_feature_importances`
* Важность перестановок
* Агрегировать DFC с помощью `est.experimental_predict_with_explanations`

Важность признаков на основе силы влияния измеряет изменение потерь при разделении на конкретный объект, в то время как важность перестановки признаков вычисляется путем оценки производительности модели при оценке, установленной путем перестановки каждого признака по одному и приписывания изменения эффективности модели с перемешанным признаком.

В целом, важность перестановки признаков предпочтительнее важности признаков на основании усиления, хотя оба метода могут быть ненадежными в ситуациях, когда прогнозируемые переменные различаются по шкале измерения или количеству категорий, а также когда признаки коррелированы ([источник](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-9-307)). Прочтите [эту статью](http://explained.ai/rf-importance/index.html), чтобы получить подробный обзор и подробное обсуждение различных типов важности признаков.

### Выжность признаков на основании усиления

Важность признаков на основе усиления встроена в оценщик TensorFlow Boosted Trees с помощью метода `est.experimental_feature_importances`.

In [ ]:
importances = est.experimental_feature_importances(normalize=True)
df_imp = pd.Series(importances)

# Визуализируем важность признаков.
N = 8
ax = (df_imp.iloc[0:N][::-1]
    .plot(kind='barh',
          color=sns_colors[0],
          title='Gain feature importances',
          figsize=(10, 6)))
ax.grid(False, axis='y')

### Средние абсолютные значения DFC
Вы также можете усреднить абсолютные значения DFC, чтобы понять влияние на глобальном уровне.

In [ ]:
# График.
dfc_mean = df_dfc.abs().mean()
N = 8
sorted_ix = dfc_mean.abs().sort_values()[-N:].index  # Среднее значение и сортировка по абсолютному значению.
ax = dfc_mean[sorted_ix].plot(kind='barh',
                       color=sns_colors[1],
                       title='Mean |directional feature contributions|',
                       figsize=(10, 6))
ax.grid(False, axis='y')

Вы также можете увидеть, как DFC меняются в зависимости от значения признака.

In [ ]:
FEATURE = 'fare'
feature = pd.Series(df_dfc[FEATURE].values, index=dfeval[FEATURE].values).sort_index()
ax = sns.regplot(feature.index.values, feature.values, lowess=True)
ax.set_ylabel('contribution')
ax.set_xlabel(FEATURE)
ax.set_xlim(0, 100)
plt.show()

### Permutation feature importance

In [ ]:
def permutation_importances(est, X_eval, y_eval, metric, features):
    """Столбец за столбцом, перемешивайте значения и наблюдайте 
    за влиянием на проверочный датасет.
    source: http://explained.ai/rf-importance/index.html
    Подобный подход можно проделать и во время тренировки. 
    См. "Важность выпадающего столбца" в статье выше."""
    baseline = metric(est, X_eval, y_eval)
    imp = []
    for col in features:
        save = X_eval[col].copy()
        X_eval[col] = np.random.permutation(X_eval[col])
        m = metric(est, X_eval, y_eval)
        X_eval[col] = save
        imp.append(baseline - m)
    return np.array(imp)

def accuracy_metric(est, X, y):
    """Точность оценки TensorFlow."""
    eval_input_fn = make_input_fn(X,
                                  y=y,
                                  shuffle=False,
                                  n_epochs=1)
    return est.evaluate(input_fn=eval_input_fn)['accuracy']
features = CATEGORICAL_COLUMNS + NUMERIC_COLUMNS
importances = permutation_importances(est, dfeval, y_eval, accuracy_metric,
                                      features)
df_imp = pd.Series(importances, index=features)

sorted_ix = df_imp.abs().sort_values().index
ax = df_imp[sorted_ix][-5:].plot(kind='barh', color=sns_colors[2], figsize=(10, 6))
ax.grid(False, axis='y')
ax.set_title('Permutation feature importance')
plt.show()

## Визуализация настройки модели

Давайте сначала смоделируем/создадим данные обучения, используя следующую формулу:


$$z=x* e^{-x^2 - y^2}$$


Где \(z\) - зависимая переменная, которую вы пытаетесь предсказать, а \(x\) и \(y\) - признаки.

In [ ]:
from numpy.random import uniform, seed
from scipy.interpolate import griddata

# Создаем ненастоящие данные
seed(0)
npts = 5000
x = uniform(-2, 2, npts)
y = uniform(-2, 2, npts)
z = x*np.exp(-x**2 - y**2)
xy = np.zeros((2,np.size(x)))
xy[0] = x
xy[1] = y
xy = xy.T

In [ ]:
# Подготавливаем данные для обучения.
df = pd.DataFrame({'x': x, 'y': y, 'z': z})

xi = np.linspace(-2.0, 2.0, 200),
yi = np.linspace(-2.1, 2.1, 210),
xi,yi = np.meshgrid(xi, yi)

df_predict = pd.DataFrame({
    'x' : xi.flatten(),
    'y' : yi.flatten(),
})
predict_shape = xi.shape

In [ ]:
def plot_contour(x, y, z, **kwargs):
  # Сетка данных.
  plt.figure(figsize=(10, 8))
  # Рисуем контур данных с координатной сеткой, отмечая точками неоднородные точки данных.
  CS = plt.contour(x, y, z, 15, linewidths=0.5, colors='k')
  CS = plt.contourf(x, y, z, 15,
                    vmax=abs(zi).max(), vmin=-abs(zi).max(), cmap='RdBu_r')
  plt.colorbar()  # Рисуем цветовую панель.
  # Рисуем точки данных.
  plt.xlim(-2, 2)
  plt.ylim(-2, 2)

Вы можете визуализировать функцию. Более темные красные цвета соответствуют бОльшим значениям функции.

In [ ]:
zi = griddata(xy, z, (xi, yi), method='linear', fill_value='0')
plot_contour(xi, yi, zi)
plt.scatter(df.x, df.y, marker='.')
plt.title('Contour on training data')
plt.show()

In [ ]:
fc = [tf.feature_column.numeric_column('x'),
      tf.feature_column.numeric_column('y')]

In [ ]:
def predict(est):
  """Прогнозы переданного оценщика."""
  predict_input_fn = lambda: tf.data.Dataset.from_tensors(dict(df_predict))
  preds = np.array([p['predictions'][0] for p in est.predict(predict_input_fn)])
  return preds.reshape(predict_shape)

Сначала давайте попробуем настроить на данные линейную модель.

In [ ]:
train_input_fn = make_input_fn(df, df.z)
est = tf.estimator.LinearRegressor(fc)
est.train(train_input_fn, max_steps=500);

In [ ]:
plot_contour(xi, yi, predict(est))

Не очень хорошая настройка. Давайте попробуем модель GBDT и постараемся понять, насколько эта модель соответствует функции.

In [ ]:
n_trees = 37 #@param {type: "slider", min: 1, max: 80, step: 1}

est = tf.estimator.BoostedTreesRegressor(fc, n_batches_per_layer=1, n_trees=n_trees)
est.train(train_input_fn, max_steps=500)
clear_output()
plot_contour(xi, yi, predict(est))
plt.text(-1.8, 2.1, '# trees: {}'.format(n_trees), color='w', backgroundcolor='black', size=20)
plt.show()

По мере увеличения количества деревьев прогнозы модели лучше аппроксимируют базовую функцию.

![](https://www.tensorflow.org/images/boosted_trees/boosted_trees_ntrees.gif)

## Заключение

В этом руководстве вы узнали, как интерпретировать модели Boosted Trees с использованием DFC и методов важности признаков. Эти методы позволяют понять, как и какие признаки влияют на прогнозы модели. Наконец, вы также получили представление о том, как модель Boosted Tree соответствует сложной функции, просмотрев результаты решений для нескольких моделей.